# Proof sizes

We run this with a modified testing environment.
The idea is to mainly depend on `sp_state_machine` crate and use a `FakeCallExecutor` and `FakeCodeExecutor` environment to avoid long compiling time.
The executor will simply check methods name and run native code.
Similarilly externalities are called directly without using host function.

In [11]:
:dep codec = { package = "parity-scale-codec", version = "3.1.3" }
:dep sp-core = "6.0.0"
:dep sp-externalities = "0.12.0"
:dep sp-state-machine = "0.12.0"
:dep sp-trie = "6.0.0"
:dep sp-runtime = "6.0.0"
:dep zstd = "0.10.0"
:dep memory-db = "0.29.0"
:dep wasmi-validation = "=0.4.0"


In [21]:
use sp_core::{traits::CodeExecutor, NativeOrEncoded};
use sp_externalities::Externalities;
use sp_core::traits::RuntimeCode;
use sp_state_machine::Backend;
use sp_state_machine::OverlayedChanges;
use sp_state_machine::TrieBackend;
use sp_runtime::traits::BlakeTwo256;
use codec::{Encode, Decode};


const STATE_VERSION: sp_core::storage::StateVersion = sp_core::storage::StateVersion::V1;

/// We use this trait so we just need to define a new instance in the following code snippets
pub trait SimpleRun: Sized + Send + Sync + Clone + 'static {
    // return block number
    fn run_runtime(&self, method: &str, ext: &mut dyn Externalities) -> (u32, Vec<u8>);
}

#[derive(Clone)]
struct FakeCodeExecutor<SR>(SR);

impl<SR: SimpleRun> sp_core::traits::ReadRuntimeVersion for FakeCodeExecutor<SR> {
   	fn read_runtime_version(
		&self,
		_wasm_code: &[u8],
		_ext: &mut dyn Externalities,
	) -> Result<Vec<u8>, String> {
        panic!("Fake implementation.")
    }
}

impl<SR: SimpleRun> CodeExecutor for FakeCodeExecutor<SR> {
	/// just some static string.
	type Error = &'static str;

	fn call<
		R: codec::Codec + PartialEq,
		NC: FnOnce() -> Result<R, Box<dyn std::error::Error + Send + Sync>> + std::panic::UnwindSafe,
	>(
		&self,
		ext: &mut dyn Externalities,
		_runtime_code: &RuntimeCode,
		method: &str,
		_data: &[u8],
		_use_native: bool,
		_native_call: Option<NC>,
	) -> (Result<NativeOrEncoded<R>, Self::Error>, bool) {
        let result = self.0.run_runtime(method, ext);
        (Ok(NativeOrEncoded::Encoded(result.encode())), true)
    }
}

fn instantiate_fake_runtime_code() -> RuntimeCode<'static> {
    RuntimeCode::empty()
}

// in memory persistence
let mut backend = sp_state_machine::new_in_mem::<sp_core::Blake2Hasher>();
let mut change_overlay = OverlayedChanges::default();

#### Example:
using this fake environment will work this way.

In [13]:
:vars

Variable,Type
backend,"TrieBackend<memory_db::MemoryDB<sp_core::Blake2Hasher, memory_db::HashKey<sp_core::Blake2Hasher>, Vec<u8>, memory_db::NoopTracker<Vec<u8>>>, sp_core::Blake2Hasher>"
change_overlay,OverlayedChanges


In [22]:

#[derive(Clone)]
struct Example1;

impl SimpleRun for Example1 {
    fn run_runtime(&self, _method: &str, ext: &mut dyn Externalities) -> (u32, Vec<u8>) {
        // block number usually is in init_block of system
        let previous_block: u32 = ext.storage(b":block_number")
            .and_then(|encoded| Decode::decode(&mut encoded.as_slice()).ok())
            .unwrap_or(0);
        let current_block = previous_block + 1;
        ext.set_storage(b":block_number".to_vec(), current_block.encode());
        
        // finalize block
        let final_root = ext.storage_root(STATE_VERSION);
        (current_block, final_root)
    }
}

let executor = FakeCodeExecutor(Example1);
let parent_state_root = sp_trie::empty_trie_root::<sp_trie::LayoutV1<BlakeTwo256>>();
let mut parent_state_root = sp_core::H256::decode(&mut &parent_state_root[..]).unwrap();


for i in 1u32..5 {
    // simulate executing a block (collator)
    let remote_root = backend.storage_root(std::iter::empty(), STATE_VERSION).0;
    let (remote_result, remote_proof) = sp_state_machine::prove_execution(
        &mut backend,
        &mut change_overlay,
        &executor,
        sp_core::testing::TaskExecutor::new(),
        "test",
        &[],
        &RuntimeCode::empty(),
    )
    .unwrap();


    // simulate verifying a block execution (pvf)
    let checked_result = sp_state_machine::execution_proof_check::<BlakeTwo256, _, _>(
        remote_root,
        remote_proof,
        &mut Default::default(),
        &executor,
        sp_core::testing::TaskExecutor::new(),
        "test",
        &[],
        &RuntimeCode::empty(),
    )
    .unwrap();

    let (block_number, root): (u32, Vec<u8>) = Decode::decode(&mut remote_result.as_slice()).unwrap();
    assert_eq!(block_number, i);
    assert_eq!(remote_result, checked_result);
    
    // flush to backend TODO consider done live
    // Note that when running try using assimilate: more correct
    let changes = change_overlay.drain_storage_changes(
        &backend,
        Default::default(), // ignore block hash
		&mut Default::default(),
		STATE_VERSION,
	)?;
	backend
		.apply_transaction(changes.transaction_storage_root, changes.transaction);
}


()

1. Write a function `run_block` for running n blocks with any `SimpleRuntime` logic. The function should return the given struct and use as input parameter a given number of blocks.

In [16]:
/// stats for a given block execution.
/// Can be displayed and compared.
#[derive(Debug, PartialEq, Eq)]
pub struct BlockExecutionStats {
    /// height
    block_number: u32,
    /// Size of proof
    proof_size: usize,
    /// Size of compact proof (lookup in susbstrate for `encode_compact`).
    compact_proof_size: usize,
    /// Size of compressed proof (from compact with zsh).
    compressed_proof_size: usize,
    /// Size of the full db. Facultative.
    full_db_size: usize,
    /// Size of the last db state (reflect more the state of a parachain with pruning). Facultative.
    last_state_db_size: usize,
}



2. Write a simple `Runtime` that writes `{block_number} world` at key `hello`.
Display result for the three first blocks.

3. Write a simple `Runtime` that appends `{block_number} world` to current value at key `hello`.
Run 100 block and display result of three first blocks, then every ten blocks.

4. Write a simple `Runtime` that writes writes `world` at key `hello {block_number}`, with BE encoding for block number (key of a trie looks way better with BE).
Run 100 block and display result of three first blocks, then every ten blocks.

5. Write a simple `Runtime` that writes writes scale encoded `("hello {block_number}", "world")` at key `blake256("hello {block_number}")`, with scale encoding (maybe compact) for block number.
Run 100 block and display result of three first blocks, then every ten blocks.

5. From 3, 4 and 5 runtimes, adds reads of n pseudo random value (use block number as seed).

Play with 5 with different parameters. What are your conclusion on the different behavior observed between 3 and 4?
Impact of using compact proof? Impact of compressing proof?

- 3 (all value inserted bellow a single key):

- 4 (all value written with their own key):

- 5 (all value written with their own key):

- Compact proof ?

- Compressing ?